In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import time
import random
import os

In [ ]:
url = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book.json"

with open(url,"r",encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 책 소개 없는 애들만
x = data.loc[data['책소개'].isna(), ['ISBN']]

In [ ]:
x.reset_index(drop=True,inplace=True)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ISBN    2998 non-null   int64
dtypes: int64(1)
memory usage: 23.6 KB


In [ ]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f"https://search.kyobobook.co.kr/search?keyword={isbn_str}&gbCode=TOT&target=total"
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면 None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음")
        return None

In [ ]:
x['url'] = x['ISBN'].apply(get_product_url_from_isbn)

ISBN 9788962515510: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000969234
ISBN 9791186492369: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9788928518357: 상품 URL -> https://product.kyobobook.co.kr/detail/S000201463394
ISBN 9791162431382: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001812721
ISBN 9788915001275: 상품 URL -> https://product.kyobobook.co.kr/detail/S000202706008
ISBN 9788962397215: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000966896
ISBN 9791137225138: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060614084
ISBN 9788952785688: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000734814
ISBN 9791137210981: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060611981
ISBN 9791190145657: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001936018
ISBN 9791127296377: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060610785
ISBN 9791160455908: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9791137267169: 

In [ ]:
# 책소개 없는 ISBN - url 데이터 저장
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
x['url'].isna().sum()

499

### 이미 존재하는 책 소개 row

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 25.2 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

#### 분류

In [ ]:
# Selenium 설정 (Colab 환경 등)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=chrome_options)

def extract_category(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    BeautifulSoup을 사용해 카테고리 정보를 추출하여
    ">국내도서>시/에세이>한국시>현대시" 형태의 문자열을 반환합니다.
    """
    # 랜덤 딜레이 (3 ~ 7초)
    delay = random.uniform(3, 7)
    time.sleep(delay)

    driver.get(url)
    # 페이지 완전 로드를 위한 대기 (필요 시 조정)
    time.sleep(6)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    ul = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(1) > ul')
    if ul:
        links = ul.select("a.intro_category_link")
        categories = [link.get_text(strip=True) for link in links]
        return ">" + ">".join(categories)
    else:
        return None

In [ ]:
# x['url']가 존재하고 x['분류']가 null인 행들만 선택
mask = x['url'].notnull() & x['분류'].isnull()
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수: {len(target_indices)}")

# 한 번에 3개씩 배치 처리 혹시 모를 트래픽
batch_size = 3
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        category_info = extract_category(url)
        x.loc[idx, '분류'] = category_info
        print(f"인덱스 {idx} | URL: {url} -> 분류: {category_info}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(7)

driver.quit()

전체 처리 대상 개수: 88
배치 1 처리 중 (총 3개)...
인덱스 16 | URL: https://product.kyobobook.co.kr/detail/S000061695386 -> 분류: >국내도서>취업/수험서>전문직자격증>행정사
인덱스 31 | URL: https://product.kyobobook.co.kr/detail/S000000902976 -> 분류: >국내도서>정치/사회>법학>소송/판례>민사소송(법)>국내도서>정치/사회>대학교재>법학>국내도서>대학교재>정치/사회/법>법학
인덱스 75 | URL: https://product.kyobobook.co.kr/detail/S000000902909 -> 분류: >국내도서>과학>교양과학>교양생물>생물이야기
배치 1 처리 완료, 다음 배치 전 7초 대기...
배치 2 처리 중 (총 3개)...
인덱스 131 | URL: https://product.kyobobook.co.kr/detail/S000061425529 -> 분류: >국내도서>취업/수험서>전문직자격증>변리사
인덱스 141 | URL: https://product.kyobobook.co.kr/detail/S000001764248 -> 분류: >국내도서>기술/공학>의학>보건학>보건의료법규>국내도서>기술/공학>대학교재>의학>국내도서>대학교재>기술공학>의학
인덱스 147 | URL: https://product.kyobobook.co.kr/detail/S000061694344 -> 분류: >국내도서>정치/사회>법학>상법>특허/상표/지식재산/저작권
배치 2 처리 완료, 다음 배치 전 7초 대기...
배치 3 처리 중 (총 3개)...
인덱스 151 | URL: https://product.kyobobook.co.kr/detail/S000201274208 -> 분류: >국내도서>경제/경영>세무/회계>양도/소득/재산세>국내도서>경제/경영>대학교재>국내도서>대학교재>경제/경영
인덱스 154 | URL: https://ebook-product.kyobobook

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


#### 책소개

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    XPath 영역
       //*[@id="scrollSpyProdInfo"]/div[4]/div[2]
    와
       //*[@id="scrollSpyProdInfo"]/div[5]/div[2]
    내에서 클래스가 "info_text fw_bold" 또는 "info_text"인 모든 div 요소(또는 해당 요소 내의 텍스트 노드)를
    순차적으로 이어붙여 하나의 문자열로 반환.

    만약 두 영역 모두에서 해당 클래스 요소가 전혀 없다면, return None
    """
    # 호출 제한 피하기
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    # 페이지가 완전히 로드될 때까지 대기 시간 1초
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 2개의 XPATH
    containers = []
    container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
    container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
    if container4:
        containers.append(container4)
    if container5:
        containers.append(container5)

    # 두 영역에서 "info_text" 관련 요소(예: "info_text fw_bold" 또는 "info_text") 두 개의 텍스트 선택 추출
    texts = []
    for cont in containers:
        divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
        if divs:
            for div in divs:
                texts.extend(list(div.stripped_strings))

    if texts:
        return " ".join(texts)
    else:
        return None

In [ ]:
# 재실행 시작 인덱스
start_index = 217

# x['url']가 존재하고 x['책소개']가 null인 행들 중에서 인덱스가 start_index 이상인 경우 선택
mask = x['url'].notnull() & x['책소개'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"남은 처리 데이터 수 (인덱스 {start_index}부터): {len(target_indices)}")

# batch 10개씩 - 호출 제한 피하기
batch_size = 10
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 1초 대기...")
    time.sleep(1)

driver.quit()

전체 처리 대상 개수 (인덱스 217부터): 1535
배치 1 처리 중 (총 10개)...
인덱스 217 | URL: https://product.kyobobook.co.kr/detail/S000200551569 -> 책소개: 2020년 초부터 전 세계적으로 확산된 코로나는 우리나라 정치, 경제, 사회, 문화 등 모든 분야에 영향을 미쳤다. 거리두기 영향과 대인접촉 기피현상으로 재택근무가 늘어나고, 각종 모임과 행사 및 회식이 줄어드는 등 큰 사회적 변화를 겪었다. 코로나 영향으로 부동산 시장도 큰 영향을 받았다.
인덱스 219 | URL: https://product.kyobobook.co.kr/detail/S000201408366 -> 책소개: None
인덱스 220 | URL: https://product.kyobobook.co.kr/detail/S000201054243 -> 책소개: 이 책은 사회복지학을 다룬 이론서이다. 알면서도 몰랐던 장애의 기초적이고 전반적인 내용을 학습할 수 있도록 구성하였다.
인덱스 228 | URL: https://product.kyobobook.co.kr/detail/S000200152387 -> 책소개: None
인덱스 229 | URL: https://product.kyobobook.co.kr/detail/S000200367257 -> 책소개: 독자대상 : 경찰공무원 시험 준비생 구성 : 이론
인덱스 230 | URL: https://product.kyobobook.co.kr/detail/S000061695353 -> 책소개: 이 책은 소박한 민속과 민가의 향기 3를 다룬 한옥건축서적이다. 소박한 민속과 민가의 향기 3에 관한 기초적이고 전반적인 내용들이 수록되어 있다.
인덱스 236 | URL: https://product.kyobobook.co.kr/detail/S000001984653 -> 책소개: 『어도비 포토샵CC』는 〈포토샵 튜닝〉, 〈환경설정〉, 〈스크레치 디스크〉, 〈사진 촬영 기법과 포토샵〉, 〈이미지 해

In [ ]:
naru = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306.csv")

In [ ]:
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47891 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


### 병합

In [ ]:
# 책소개 채기
if '책소개' in naru.columns and '책소개' in x.columns:
    # ISBN을 기준으로 x를 딕셔너리로 변환
    isbn_to_intro = x.set_index('ISBN')['책소개'].dropna().to_dict()

    # '책소개'가 None (또는 NaN)인 경우 x의 '책소개' 값으로 채우기
    naru['책소개'] = naru.apply(lambda row: isbn_to_intro.get(row['ISBN'], row['책소개']) if pd.isna(row['책소개']) else row['책소개'], axis=1)
else:
    print("'분류' column X")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49255 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
naru['책소개'] = naru.apply(lambda x: x['간략소개'] if pd.isna(x['책소개']) and pd.notna(x['간략소개']) else x['책소개'], axis=1)

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49616 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 분류 채우기
if '분류' in naru.columns and '분류' in x.columns:
    # ISBN 기준 x 딕셔너리 변환
    isbn_to_intro = x.set_index('ISBN')['분류'].dropna().to_dict()

    # '분류'가 None이면, x의 '분류' 값으로 채우기
    naru['분류'] = naru.apply(lambda row: isbn_to_intro.get(row['ISBN'], row['분류']) if pd.isna(row['분류']) else row['분류'], axis=1)
else:
    print("'분류' column X")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49243 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49255 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"
pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
# original
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2895 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
# 1차 데이터 filled
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49616 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 최종 데이터 저장
naru.to_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled).csv", index=False)

* //*[@id="scrollSpyProdInfo"]/div[4]/div[3]/div/text()[1]
    * 해당 주소로 758개 다시 받아오기
* 출판사리뷰(서평) + 추천사
    * 존재하는 것만 빨리빨리 보고 넘어가게 코드 짜기
* 목차

In [ ]:
# 3차 data filled를 위한 2차 처리 데이터 불러오기
data = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled)_v2.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49626 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from lxml import etree

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    - 만약 <div class="info_text"> 요소가 존재하면,
      XPath 경로 '//*[@id="scrollSpyProdInfo"]/div[2]/div[2]/div//text()'를 통해
      직접 텍스트 노드와 자식 노드(숫자 인덱스로 표현된 경우 포함)의 텍스트를 모두 추출 후, 이어붙여서 반환.
    - 기존 방식(컨테이너 div[4], div[5] 등)으로 텍스트를 추출. 없으면, return None
    """
    # 호출 트레픽 완환을 위한 대기
    time.sleep(random.uniform(1, 2))
    driver.get(url)
    # 동적 셀레니움 - 페이지가 완전히 로드될 때까지 1초 대기
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # <div class="info_text">가 존재하면, 1차 데이터 수집과 다른, 새로운 XPath 경로 사용
    if soup.find("div", class_="info_text"):
        tree = etree.HTML(html)
        # 직접 텍스트 노드와 하위 요소의 텍스트 모두 추출 (숫자 인덱스가 붙은 경우도 포함)
        texts = tree.xpath('//*[@id="scrollSpyProdInfo"]/div[2]/div[2]/div//text()')
        texts = [t.strip() for t in texts if t.strip()]
        return " ".join(texts) if texts else None
    else:
        # 혹시 모르니까 기존 방식: div[4], div[5] 및 추가 영역에서도 텍스트 추출
        containers = []
        container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
        container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
        if container4:
            containers.append(container4)
        if container5:
            containers.append(container5)
        container_extra = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(3)')
        if container_extra:
            containers.append(container_extra)

        texts = []
        for cont in containers:
            divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
            if divs:
                for div in divs:
                    texts.extend(list(div.stripped_strings))
            else:
                texts.extend(list(cont.stripped_strings))
        return " ".join(texts) if texts else None

In [ ]:
# 재실행 시작 인덱스 - 이미 한번 수집된 데이터, x_url 을 불러왔으므로 0번으로 시작
start_index = 0

# x['url']이 존재하고 x['책소개']가 null인 행들 중에서 인덱스가 start_index 이상인 경우 선택
mask = x['url'].notnull() & x['책소개'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

# batch 10개씩 - 호출 제한 피하기
batch_size = 10
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 1초 대기...")
    time.sleep(1)

driver.quit()

전체 처리 대상 개수 (인덱스 0부터): 302
배치 1 처리 중 (총 10개)...
인덱스 0 | URL: https://product.kyobobook.co.kr/detail/S000000969234 -> 책소개: 도서 English Literature and Reading 김재균 14,400 원 English Linguistics 영어학 Basic 김재균 13,500 원 전공영어 English Linguistics(Basic) 김재균 13,500 원 전공영어: 일반영어 문학 김재균 13,500 원 English Reading and Literature New Edition(전공영어) 김재균 13,500 원 English Linguistics 김재균 14,400 원 The English Language Teaching(전공영어) 김재균 14,400 원 2026 중등교원 임용시험대비 정치학 김현중 21,600 원 2026 중등교원 임용시험대비 사회문화 이웅재 21,600 원 2026 이선화 교육학 잇키(IT-KEY) 핵심 키워드 암기 자료 이선화 9,900 원 문영은 전공가정 주영역 문영은 13,500 원 2026 서진 끈내주는 기출문제집(영역별) 1 세트 서진. 정조이 46,800 원 문영은 전공가정 자원 소비영역 문영은 13,500 원 2026 권은성 ZOOM 전공체육 운동생리학 트레이닝론 권은성 13,500 원 2026 하이패스 교직논술 기본편+기출·실전편 세트 조학규 35,100 원
인덱스 6 | URL: https://product.kyobobook.co.kr/detail/S000060614084 -> 책소개: None
인덱스 8 | URL: https://product.kyobobook.co.kr/detail/S000060611981 -> 책소개: None
인덱스 10 | URL: https://product.kyobobook.co.kr/detail/S000060610785 -> 책소개: None
인덱스 12 | URL: https://produc

In [ ]:
# 잘못 뽑힌 데이터 - [0,29,43,110,174,177,216,219,228,264,266,287,294,307,311,388,464,469,473,478,485,492,493,497,518,526,527,533,627,653,695,827,861,870,907,960,1076,1100,1207,1252,1255,1259,1322,1472,1475,1509,1529,1691,1761,1780,1781,1813,1815,1894,2043,2059,2180,2214,2220,2234,2273,2534,2609,2632,2640,2641,2744,2792,2805,2838,2884,2895,2988,2990]

In [ ]:
x['책소개']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2417 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
indices_to_replace = [0,29,43,110,174,177,216,219,228,264,266,287,294,307,311,388,464,469,473,478,485,492,493,497,518,526,527,533,627,653,695,827,861,870,907,960,1076,1100,1207,1252,1255,1259,1322,1472,1475,1509,1529,1691,1761,1780,1781,1813,1815,1894,2043,2059,2180,2214,2220,2234,2273,2534,2609,2632,2640,2641,2744,2792,2805,2838,2884,2895,2988,2990]
x.loc[indices_to_replace, '책소개'] = None

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2343 non-null   object
 3   출판사리뷰   374 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
# 잘못 뽑힌 데이터 중 추가할만한 text가 존재하는 데이터 - [485,492,497,518,526,527,627,653,861,907,960,1076,1207,1255,1259,1472,1475,1813,1894,2214,2220,2234,2273,2534,2744,2805,2895,2988]

In [ ]:
# 수동으로 집어넣기
x.iloc[485,2] = "배틀 도감의 원조, 베스트셀러 “최강” 시리즈 9탄! 이번엔 신화 세계의 신, 몬스터, 악마가 맞붙는다! 세계 각지의 신화와 전설 속의 신, 악마, 몬스터 중 최강을 가린다. 옛날 사람들은 사람의 힘으로 통제할 수 없고 이해할 수 없는 여러 자연현상을 어떤 초월적인 존재가 일으킨 것이라고 믿었습니다. 그리고 그런 존재가 ‘신’으로 구현되었죠. 인류의 문명에서 신이 등장한 시기에 신에 대적하는 존재로 ‘악마’도 등장했습니다. 또한 드래곤과 같은 거대하고 강한 힘을 가진 ‘몬스터’도 신화와 전설에 나오게 되었습니다. 세계 각지의 신화와 전설 속의 이런 막강한 힘을 가진 존재들 중 진짜 최강자는 누구일까? 바로 이 궁금증을 해결하기 위해 〈최강 신화왕〉이 나왔습니다. 이 책에는 16신과 8마리의 악마·몬스터가 등장해 대결을 벌입니다. 막강한 힘과 신묘한 무기, 탁월한 전술로 상대를 압도하고 승리를 거머쥐어 최강 신화왕의 자리에 오르는 최후의 1인은 과연 누구일까요? 압도적인 스케일의 박진감 넘치는 배틀을 따라가면서 그 답을 찾아봅시다."

In [ ]:
x.iloc[485,2]

'배틀 도감의 원조, 베스트셀러 “최강” 시리즈 9탄! 이번엔 신화 세계의 신, 몬스터, 악마가 맞붙는다! 세계 각지의 신화와 전설 속의 신, 악마, 몬스터 중 최강을 가린다. 옛날 사람들은 사람의 힘으로 통제할 수 없고 이해할 수 없는 여러 자연현상을 어떤 초월적인 존재가 일으킨 것이라고 믿었습니다. 그리고 그런 존재가 ‘신’으로 구현되었죠. 인류의 문명에서 신이 등장한 시기에 신에 대적하는 존재로 ‘악마’도 등장했습니다. 또한 드래곤과 같은 거대하고 강한 힘을 가진 ‘몬스터’도 신화와 전설에 나오게 되었습니다. 세계 각지의 신화와 전설 속의 이런 막강한 힘을 가진 존재들 중 진짜 최강자는 누구일까? 바로 이 궁금증을 해결하기 위해 〈최강 신화왕〉이 나왔습니다. 이 책에는 16신과 8마리의 악마·몬스터가 등장해 대결을 벌입니다. 막강한 힘과 신묘한 무기, 탁월한 전술로 상대를 압도하고 승리를 거머쥐어 최강 신화왕의 자리에 오르는 최후의 1인은 과연 누구일까요? 압도적인 스케일의 박진감 넘치는 배틀을 따라가면서 그 답을 찾아봅시다.'

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2371 non-null   object
 3   출판사리뷰   374 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49626 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

### 최종 책소개 & 출판사 리뷰 데이터 merge

In [ ]:
# 간략소개 - 책소개 대체로 인해 책소개 == 제목이 되버린 행 None 처리
data.loc[data['제목'] == data['책소개'], '책소개'] = np.nan

In [ ]:
# 약 750개의 None
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49263 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# ISBN을 기준으로 data에 x를 merge
merged = data.merge(x[['ISBN', '책소개']], on='ISBN', how='left', suffixes=('_data', '_x'))

# data['책소개']가 NaN인 경우 x['책소개'] 값으로 대체
data.loc[data['책소개'].isna(), '책소개'] = merged.loc[data['책소개'].isna(), '책소개_x']

In [ ]:
# 간략소개 - 책소개 대체로 인해 책소개 == 제목이 되버린 데이터 중, url로 불러와 채운 데이터 약 130개
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49394 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
z = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv")
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20844 entries, 0 to 20843
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         20844 non-null  int64 
 1   product_url  18937 non-null  object
 2   추천사          523 non-null    object
 3   출판사리뷰        626 non-null    object
dtypes: int64(1), object(3)
memory usage: 651.5+ KB


In [ ]:
both_exist = ((z['추천사'].notna()) & (z['출판사리뷰'].notna())).sum()
only_recommendation = ((z['추천사'].notna()) & (z['출판사리뷰'].isna())).sum()
only_review = ((z['추천사'].isna()) & (z['출판사리뷰'].notna())).sum()
both_missing = ((z['추천사'].isna()) & (z['출판사리뷰'].isna())).sum()

# 둘다 있음, 추천사만 있음, 리뷰만 있음, 둘 다 없음
both_exist, only_recommendation, only_review, both_missing

(60, 463, 566, 19755)

In [ ]:
# ISBN을 기준으로 data에 z merge
merged = data.merge(z[['ISBN', '출판사리뷰', '추천사']], on='ISBN', how='left', suffixes=('_data', '_z'))

# data['출판사리뷰']가 NaN인 경우 z['출판사리뷰'] 값으로 대체
data.loc[data['출판사리뷰'].isna(), '출판사리뷰'] = merged.loc[data['출판사리뷰'].isna(), '출판사리뷰_z']

# data에 '추천사' 컬럼 추가 후 z['추천사'] 값 삽입
data['추천사'] = merged['추천사']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49394 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29782 non-null  object 
 19  INSE

In [ ]:
data.to_csv('/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250310_update(filled)_v3.csv', index=False, encoding='utf-8-sig')

In [ ]:
merged = data[['ISBN', '책소개']].merge(x[['ISBN', '책소개']], on='ISBN', how='inner', suffixes=('_data', '_x'))

# 결과 출력
merged

,ISBN,책소개_data,책소개_x
0,9788962515510,NaN,None
1,9791186492369,재미있는 구성과 활동을 통해 아이들이 지루하지 않고 즐겁게 학습하는 습관을 길러주며...,NaN
2,9788928518357,『이재수 실기』는 1901년 제주도에서 발생했던 ‘이재수의 난’의 전 과정을 소설 ...,『이재수 실기』는 1901년 제주도에서 발생했던 ‘이재수의 난’의 전 과정을 소설 ...
3,9791162431382,"임영석 시집 『나, 이제부터 삐딱하게 살기로 했다』는 크게 5부로 나누어져 있으며 ...","임영석 시집 『나, 이제부터 삐딱하게 살기로 했다』는 크게 5부로 나누어져 있으며 ..."
4,9788915001275,삼성출판사의 글로벌 프로젝트 BLUE BIRD! 블루버드는 전권을 해외 정상 일러스...,삼성출판사의 글로벌 프로젝트 BLUE BIRD! 블루버드는 전권을 해외 정상 일러스...
...,...,...,...
2993,9791169501262,이 책은 관공서중심의 공공디자인복합건축물을 다룬 건축디자인 서적이다. 관공서중심의 ...,이 책은 관공서중심의 공공디자인복합건축물을 다룬 건축디자인 서적이다. 관공서중심의 ...
2994,9788968472398,독자대상 : 간호사 시험 준비생 구성 : 이론 + 문제 등,독자대상 : 간호사 시험 준비생 구성 : 이론 + 문제 등
2995,9788956768861,"이 책은 운동역학에 대해 다룬 도서이며 〈운동역학의 개요〉, 〈운동역학의 이해〉, ...","이 책은 운동역학에 대해 다룬 도서이며 〈운동역학의 개요〉, 〈운동역학의 이해〉, ..."
2996,9791136426611,사내가 무림에 처음 등장했을 때 세상은 염왕(閻王)이라 부르며 두려워했다. 사내가 ...,사내가 무림에 처음 등장했을 때 세상은 염왕(閻王)이라 부르며 두려워했다. 사내가 ...


In [ ]:
filtered = merged[merged['책소개_data'].isna() & merged['책소개_x'].notna()]

# 결과 출력
filtered

,ISBN,책소개_data,책소개_x
34,9791141013424,NaN,-
73,9791192786049,NaN,1. 아가페사랑경영관점 사랑경영학은 사랑과 경영학의 합성어이다. 조화를 이루기가 힘...
82,9791192786124,NaN,1. 아가페사랑경영관점 사랑경영학은 사랑과 경영학의 합성어이다. 조화를 이루기가 힘...
100,9791141013639,NaN,이 책의 주제는 ‘연결’이다. 모든 사건은 연결로부터 시작된다. 음악적 사건도 음과...
107,9791137298811,NaN,"책소개 이 책은 시 형식의 새로운 자서전적 에세이로써 돈 때문에 꿈을 포기하거나, ..."
...,...,...,...
2933,9791141005757,NaN,"작가의 60~70년대의 어린 시절을 보면서 시대상을 볼 수 있고, 결혼해서 남편의 ..."
2956,9791141003470,NaN,"처음으로 운문에 빠져 적어보았습니다. 어떤 부분들은 공감도 갈 것이고, 어떤 부분들..."
2960,9791141018474,NaN,매일 무엇을 먹을지 고민하는 여러분들을 위해 책을 펼치면 한 가지의 메뉴를 추천해 ...
2975,9791141024109,NaN,드디어 코비드 19가 끝나간다. 3년 동안의 팬데믹 상황이 이제 막바지에 이르러 마...


In [ ]:
# x에서 x['책소개']가 존재하는 행의 ISBN과 책소개를 매핑하는 딕셔너리 생성
isbn_to_intro = (
    x.loc[x['책소개'].notnull(), ['ISBN', '책소개']]
    .drop_duplicates('ISBN')
    .set_index('ISBN')['책소개']
)

# data에서 '책소개'가 None인 행 중, 해당 ISBN이 x에 존재하는 경우 찾기
mask = data['책소개'].isnull() & data['ISBN'].isin(isbn_to_intro.index)

# 매핑된 값을 data['책소개']에 대입
data.loc[mask, '책소개'] = data.loc[mask, 'ISBN'].map(isbn_to_intro)

### 출판사 서평 - url

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled).csv")

In [ ]:
# 출판사리뷰 없는 애들만
x = data.loc[data['출판사리뷰'].isna(), ['ISBN']]

In [ ]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f"https://search.kyobobook.co.kr/search?keyword={isbn_str}&gbCode=TOT&target=total"
    response = requests.get(search_url)
    if response.status_code != 200: # 상태 양호 코드
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser') # html 파서

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 있으면 받고 없으면, None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음...")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면, None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음...")
        return None

In [ ]:
# 저장 파일 경로 설정
save_dir = "/content/drive/MyDrive/aiffel_final_project/data_renew"
save_file = os.path.join(save_dir, "x_review.csv")

# 이미 저장된 결과가 있는지 확인 (끊긴 부분부터 재시작)
if os.path.exists(save_file):
    df_saved = pd.read_csv(save_file)
    processed_isbns = set(df_saved["ISBN"].astype(str))
    print(f"저장된 결과가 있습니다. 처리된 ISBN 수: {len(processed_isbns)}")
else:
    df_saved = pd.DataFrame(columns=["ISBN", "product_url"])
    processed_isbns = set()

# 이미 처리된 결과를 포함하여 결과 DataFrame 초기화
results = df_saved.copy()
batch_count = 0  # 이번 실행에서 새로 처리한 건수

# 전체 x DataFrame의 ISBN에 대해 반복 (이미 처리된 것은 건너뜀)
for idx, row in x.iterrows():
    isbn = row["ISBN"]
    if str(isbn) in processed_isbns:
        continue

    url = get_product_url_from_isbn(isbn)
    # 새로운 행 : product_url 생성 후, 받아온 url merge
    new_row = pd.DataFrame([{"ISBN": isbn, "product_url": url}])
    results = pd.concat([results, new_row], ignore_index=True)
    processed_isbns.add(str(isbn))
    batch_count += 1

    # 300건마다 저장
    if batch_count % 300 == 0:
        results.to_csv(save_file, index=False)
        print(f"새로 처리한 {batch_count}건 저장 완료. (전체 저장된 건수: {len(results)})")
        time.sleep(1)  # 호출 대기 1초

results.to_csv(save_file, index=False)
print("모든 ISBN 처리 & 저장 완료")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
ISBN 9791159629655: URL -> https://product.kyobobook.co.kr/detail/S000001782446
ISBN 9791163899525: URL -> https://product.kyobobook.co.kr/detail/S000001823740
ISBN 9788967641757: URL -> https://product.kyobobook.co.kr/detail/S000001048840
ISBN 9791155775844: URL -> https://product.kyobobook.co.kr/detail/S000001728969
ISBN 9791164410507: URL -> https://product.kyobobook.co.kr/detail/S000001828758
ISBN 9791159422232: URL -> https://product.kyobobook.co.kr/detail/S000001778123
ISBN 9791136264121: URL -> https://product.kyobobook.co.kr/detail/S000001712203
ISBN 9788989625315: URL -> https://product.kyobobook.co.kr/detail/S000001428808
ISBN 9791164451968: URL -> https://product.kyobobook.co.kr/detail/S000001829837
ISBN 9791138012072: URL -> https://product.kyobobook.co.kr/detail/S000001716127
ISBN 9791136234704: URL -> https://product.kyobobook.co.kr/detail/S000001711212
ISBN 9791168488670: URL -> https://product.kyobobook.co.kr/detail/S000001853613
ISBN

### 출판사리뷰 - text

In [ ]:
# 불러오기
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_review.csv")

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 33.7 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-

In [ ]:
import os
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

save_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv"

if os.path.exists(save_path):
    x = pd.read_csv(save_path)
    print(f"CSV 파일 로드 성공: {save_path}")
else:
    raise FileNotFoundError(f"CSV 파일이 {save_path}에 존재X, 파일불러오기 실패")

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_details(url):
    """
    주어진 URL에 접속 후, '추천사'와 '출판사리뷰' 데이터를 추출

    추천사:
      - 'product_detail_area book_recommend' 클래스가 존재하면,
        <ul class="recommend_list"> 내부의 각 <li class="recommend_item">에서
        추천자 : 추천 텍스트 형식으로 추출 후, 연결.
      - 없으면 return None

    출판사리뷰:
      - 'product_detail_area book_publish_review' 클래스가 존재하면,
        내부의 <p class="info_text">의 텍스트를 추출.
      - 없으면 return None
    """
    # 랜덤 딜레이 (1 ~ 2초) - API 호출 제한
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    time.sleep(1)  # 동적 페이지 로드 대기 - 1초
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 추천사 추출
    recommend_text = None
    recommend_section = soup.find("div", class_="product_detail_area book_recommend")
    if recommend_section:
        ul = recommend_section.find("ul", class_="recommend_list")
        if ul:
            rec_items = ul.find_all("li", class_="recommend_item")
            rec_list = []
            for item in rec_items:
                source_tag = item.find("a", class_="title_heading fc_spot type_link")
                source = source_tag.get_text(strip=True) if source_tag else ""
                info_tag = item.find("p", class_="info_text")
                info = info_tag.get_text(strip=True) if info_tag else ""
                if source or info:
                    rec_list.append(f"{source} : {info}")
            if rec_list:
                recommend_text = "\n".join(rec_list)

    # 출판사리뷰 추출
    publish_review_text = None
    publish_section = soup.find("div", class_="product_detail_area book_publish_review")
    if publish_section:
        p_tag = publish_section.find("p", class_="info_text")
        if p_tag:
            publish_review_text = p_tag.get_text(separator="\n", strip=True)

    return recommend_text, publish_review_text

CSV 파일 로드됨: /content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv


In [ ]:
# 시작 인덱스
start_index = 19423

# columns 추가
if '추천사' not in x.columns:
    x['추천사'] = None
if '출판사리뷰' not in x.columns:
    x['출판사리뷰'] = None

# product_url이 존재하고, '추천사'와 '출판사리뷰'가 None인 행 선택 (start_index 이상)
mask = x['product_url'].notnull() & x['추천사'].isnull() & x['출판사리뷰'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"남은 처리 개수 (인덱스 {start_index}부터): {len(target_indices)}")

# 10개씩 처리 - API 호출 제한
batch_size = 10
total = len(target_indices)
processed_count = 0

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'product_url']
        rec_text, pub_review = extract_book_details(url)
        x.loc[idx, '추천사'] = rec_text
        x.loc[idx, '출판사리뷰'] = pub_review
        processed_count += 1
        print(f"인덱스 {idx} | URL: {url} -> 추천사: {rec_text}, 출판사리뷰: {pub_review}")

        # 100번마다 CSV로 파일 저장
        if processed_count % 100 == 0:
            x.to_csv(save_path, index=False)
            print(f"Processed {processed_count} rows. CSV 저장됨: {save_path}")

    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 실행중...")
    time.sleep(1)

x.to_csv(save_path, index=False)
print(f"모든 데이터 처리 & 파일 저장 완료: {save_path}")

driver.quit()

전체 처리 대상 개수 (인덱스 19423부터): 1298
배치 1 처리 중 (총 10개)...
인덱스 19423 | URL: https://product.kyobobook.co.kr/detail/S000001941965 -> 추천사: None, 출판사리뷰: None
인덱스 19424 | URL: https://product.kyobobook.co.kr/detail/S000001821615 -> 추천사: None, 출판사리뷰: None
인덱스 19425 | URL: https://product.kyobobook.co.kr/detail/S000061583902 -> 추천사: None, 출판사리뷰: None
인덱스 19426 | URL: https://product.kyobobook.co.kr/detail/S000000504833 -> 추천사: None, 출판사리뷰: None
인덱스 19427 | URL: https://product.kyobobook.co.kr/detail/S000000766963 -> 추천사: None, 출판사리뷰: None
인덱스 19428 | URL: https://product.kyobobook.co.kr/detail/S000061449359 -> 추천사: None, 출판사리뷰: None
인덱스 19429 | URL: https://product.kyobobook.co.kr/detail/S000001795575 -> 추천사: None, 출판사리뷰: None
인덱스 19431 | URL: https://product.kyobobook.co.kr/detail/S000001711076 -> 추천사: None, 출판사리뷰: None
인덱스 19432 | URL: https://product.kyobobook.co.kr/detail/S000200765449 -> 추천사: None, 출판사리뷰: None
인덱스 19433 | URL: https://product.kyobobook.co.kr/detail/S000200408780 -> 추천사: None,